In [11]:
mutable struct Driver
    car_length::Float64
    driver_skill::Float64
    parking_duration::Int
    distance_tolerance::Float64
    parking_policy::Float64
end

function create_driver(mean_len, len_dev, skill_mean, skill_dev, park_time_mean, park_time_dev, dist_tolerance_mean, dist_tolerance_dev, policy_mean, policy_dev)
    car_length = round(mean_len .+ len_dev * randn(), digits=1)
    if car_length < 3
        car_length = 3
    elseif car_length > 5.5
        car_length = 5.5
    end
    driver_skill = max(0, skill_mean + skill_dev * randn())
    parking_duration = max(1, round(park_time_mean + park_time_dev * randn()))
    distance_tolerance = max(0, dist_tolerance_mean + dist_tolerance_dev * randn())
    parking_policy = max(0, policy_mean + policy_dev * randn())
    return Driver(car_length, driver_skill, parking_duration, distance_tolerance, parking_policy)
end

function simulate_car_parking(num_drivers, mean_len, len_dev, skill_mean, skill_dev, park_time_mean, park_time_dev, dist_tolerance_mean, dist_tolerance_dev, policy_mean, policy_dev)
    drivers = [create_driver(mean_len, len_dev, skill_mean, skill_dev, park_time_mean, park_time_dev, dist_tolerance_mean, dist_tolerance_dev, policy_mean, policy_dev) for i in 1:num_drivers]

    curb_length = 100.0
    available_spaces = [(0.0, curb_length)]  # (start, end) positions of available spaces on the curb

    parked_cars = []
    tick_counter = 0

    while true
        tick_counter += 1
        println("Tick: ", tick_counter)

        if isempty(drivers)
            break
        end

        driver = pop!(drivers)
        car_length = driver.car_length
        driver_skill = driver.driver_skill
        parking_duration = driver.parking_duration
        distance_tolerance = driver.distance_tolerance
        parking_policy = driver.parking_policy

        spot_found = false
        spot_index = 0

        # Find available space that fits car_length * driver_skill
        for (i, (start, stop)) in enumerate(available_spaces)
            spot_length = stop - start
            if spot_length >= car_length * driver_skill
                spot_found = true
                spot_index = i
                break
            end
        end

        if spot_found
            # Occupy the available spot
            (start, stop) = available_spaces[spot_index]
            occupied_length = car_length * driver_skill

            if occupied_length < (stop - start)
                # Allocate extra space according to parking policy
                extra_space = (stop - start) - occupied_length
                allocation = occupied_length + extra_space * parking_policy

                if allocation + start <= stop && allocation >= car_length * driver_skill
                    occupied_length = allocation
                end
            end

            println("Driver parked from ", start, " to ", start + occupied_length)
            push!(parked_cars, (start, start + occupied_length))

            if occupied_length < (stop - start)
                # Split the available space into occupied and remaining space
                new_start = start + occupied_length
                new_stop = stop
                available_spaces[spot_index] = (new_start, new_stop)
            else
                # Remove the fully occupied space
                splice!(available_spaces, spot_index)
            end

            # Update parking duration for the driver
            if parking_duration > 1
                driver.parking_duration -= 1
                push!(drivers, driver)
            else
                println("Driver vacated parking spot")
            end
        else
            println("No available spot for the driver")
        end
    end

    println("All drivers have vacated")
end

simulate_car_parking (generic function with 1 method)

In [12]:
# Example usage
mean_len = 4.25
len_dev = 0.5
skill_mean = 0.8
skill_dev = 0.1
park_time_mean = 5
park_time_dev = 2
dist_tolerance_mean = 2
dist_tolerance_dev = 0.5
policy_mean = 0.5
policy_dev = 0.2

num_drivers = 10
simulate_car_parking(num_drivers, mean_len, len_dev, skill_mean, skill_dev, park_time_mean, park_time_dev, dist_tolerance_mean, dist_tolerance_dev, policy_mean, policy_dev)

Tick: 1
Spot Index: 1
Driver parked from 0.0 to 17.93906691891003
Tick: 2
Spot Index: 1
Driver parked from 17.93906691891003 to 33.19952807836124
Tick: 3
Spot Index: 1
Driver parked from 33.19952807836124 to 46.181344571482875
Tick: 4
Spot Index: 1
Driver parked from 46.181344571482875 to 57.224756543126425
Tick: 5
Spot Index: 1
Driver parked from 57.224756543126425 to 66.61920053638873
Driver vacated parking spot
Tick: 6
Spot Index: 1
Driver parked from 66.61920053638873 to 89.81074011670911
Tick: 7
Spot Index: 1
Driver parked from 89.81074011670911 to 97.63817009994972
Tick: 8
Spot Index: 0
No available spot for the driver
Tick: 9
Spot Index: 0
No available spot for the driver
Tick: 10
Spot Index: 0
No available spot for the driver
Tick: 11
Spot Index: 0
No available spot for the driver
Tick: 12
Spot Index: 0
No available spot for the driver
Tick: 13
Spot Index: 0
No available spot for the driver
Tick: 14
Spot Index: 0
No available spot for the driver
Tick: 15
Spot Index: 0
No availa

In [ ]:
function plot_cars(min_value, max_value)
    plot(size=(800, 100), legend=false)
    xticks!(min_value:10:max_value)  # Set the tick locations to match the number line
    yaxis!(:off)  # Turn off the y-axis
    plot!([min_value, max_value], [0, 0], linewidth=3, color=:black)  # Plot the number line
    scatter!([min_value, max_value], [0, 0], markersize=6, marker=:circle, color=:black)  # Add markers at the ends
    xlims!(min_value-1, max_value+1)  # Set the x-axis limits slightly wider than the number line range
    ylims!(-0.5, 0.5)  # Set the y-axis limits to center the number line
end

plot_number_line(0, 200)


In [ ]:
using Graphs
using Random

function create_random_graph(num_vertices, num_edges)
    if num_vertices < 2
        error("The number of vertices must be at least 2.")
    end

    if num_edges < num_vertices - 1 || num_edges > 4 * num_vertices
        error("The number of edges must be at least (num_vertices - 1) and at most 4 times the number of vertices.")
    end

    g = SimpleGraph()
    vertices = collect(1:num_vertices)
    edges_added = 0

    # Connect the first two vertices
    add_edge!(g, 1, 2)
    edges_added += 1

    while edges_added < num_edges
        v = rand(vertices)
        u = rand(vertices)

        # Check if v and u are different and there is no edge between them
        if v != u && !has_edge(g, v, u)
            add_edge!(g, v, u)
            edges_added += 1
        end
    end

    return g
end
